In [33]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from os.path import exists
from pydub import AudioSegment
import random

import sys
sys.path.insert(0, "../Src/")

%autosave 5

Autosaving every 5 seconds


In [ ]:
#dataSet = pd.read_csv('../Data/train_data.csv')
#dataSet[dataSet['speaker_id'] == 'MMDM0']

  Obtaining dependency information for Voice-Cloning from https://files.pythonhosted.org/packages/c8/e4/a2967e9088f30cd277e796d2f60f09e960b776fade0f1076dcae24c5bb91/Voice_Cloning-0.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for inflect==5.3.0 from https://files.pythonhosted.org/packages/4f/a8/031641ad73a1bd1a9932261a6193864556172b333dde263fed8b5a0940cf/inflect-5.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for librosa==0.8.1 from https://files.pythonhosted.org/packages/54/19/a0e2bdc94bc0d1555e4f9bc4099a0751da83fa6e1e6157ec005564f8a98a/librosa-0.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for sounddevice from https://files.pythonhosted.org/packages/6f/f6/6703fe7cf3d7b7279040c792aeec6334e7305956aba4a80f23e62c8fdc44/sounddevice-0.5.1-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl.metadata
  Obtaining dependency information for SoundFile from https://files.pythonhosted.org/packages/a0/e5/58fd1a8d7b26fc113af244f966ee3aec

In [35]:
def readCsv(dataset):
    return pd.read_csv(f'../Data/{dataset}_data.csv')

In [36]:
def getSpeakers(df):
    speakerIds = df['speaker_id']
    return list(set(speakerIds))

In [37]:
def getFilesBySpeaker(df,speakerId):
    return df[df['speaker_id']==speakerId]

In [38]:
def concatenateAudio(speakerId, speakerDF):
    if speakerDF.empty:
        print(f"Empty DataFrame for speaker {speakerId}. Skipping.")
        return

    finalAudioFile = f'../Data/concatenatedInputs/{speakerId}.wav'
    audioData = speakerDF[speakerDF['filename'].str.endswith('.wav', na=False)]

    if audioData.empty:
        print(f"No .wav files for speaker {speakerId}. Skipping.")
        return

    audioFileList = audioData['path_from_data_dir']

    if not exists(finalAudioFile):
        concat_audio = AudioSegment.empty()
    else:
        concat_audio = AudioSegment.from_wav(finalAudioFile)
    for audioFile in audioFileList:
        try:
            audio = AudioSegment.from_wav(f'../Data/data/{audioFile}') + AudioSegment.silent(duration=1000)
            concat_audio += audio
        except Exception as e:
            print(f"Failed to load {audioFile}: {e}")

    if len(concat_audio) > 0:
        concat_audio.export(finalAudioFile, format='wav')
    else:
        print(f"No valid audio for speaker {speakerId}, nothing exported.")
    

In [39]:
import os


def main():
    trainDF = readCsv('train')
    speakers = getSpeakers(trainDF)
    if not exists('../Data/concatenatedInputs'):
        os.makedirs('../Data/concatenatedInputs')
        for speaker in speakers:
            speakerDF = getFilesBySpeaker(trainDF,speaker)
            concatenateAudio(speaker,speakerDF)
    

In [40]:
if __name__ == '__main__':
    main()